<a href="https://colab.research.google.com/github/MrToffel/colab-KI/blob/main/Text_KI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KI entscheidet ob ein Text richtig oder eine sinnlose Zeichenkombination ist<br>


1: Trainingstext laden

In [1]:
!wget https://raw.githubusercontent.com/MrToffel/colab-KI/main/in.txt       #trainingstext

--2021-03-19 15:56:42--  https://raw.githubusercontent.com/MrToffel/colab-KI/main/in.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 446043 (436K) [text/plain]
Saving to: ‘in.txt’

in.txt              100%[===================>] 435.59K  --.-KB/s    in 0.02s   

2021-03-19 15:56:42 (23.3 MB/s) - ‘in.txt’ saved [446043/446043]



2: Imports ausführen

In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

3: Textdatei lesen und verschiedenen Buchstaben eine Id zuweisen


In [3]:
raw_text=str(open("in.txt").read()) #alle zeichen hier rein
raw_text2="QWERTZUIOPÜ*'ASDFGHJKLÖÄYXCVBNM;:_qwertzuiopü+#asdfghjklöäyxcvbnm,.-<>|@1234567890ß!\"§$%&/()=? \n»«~+*\\—½{}[]è" #alle zeichen hier rein
chars = sorted(list(set(raw_text2)))
print(chars)
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
n_vocab = len(chars)
print ("Total Vocab: ", n_vocab)
n_chars = len(raw_text)
print ("Total Characters: ", n_chars)


['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '§', '«', '»', '½', 'Ä', 'Ö', 'Ü', 'ß', 'ä', 'è', 'ö', 'ü', '—']
Total Vocab:  107
Total Characters:  437677


4: Text in kleine Ausschnitte zerhacken mit der Länge seq_length und Eingabe-Ausgabe-Paare generieren. <br>
Um dem Modell auch falschen Text zu zeigen wird für jeden wahren String auch ein Falscher mit zufälligen Zeichen aus chars erstellt. <br>So bekommt das Modell die gleiche Zahl an richtigem und falschem Text zum üben

In [4]:
seq_length = 5
dataX = []
dataY = []
for i in range(n_chars-seq_length):
    data=raw_text[i:i+seq_length];
    random_string = ''
    for _ in range(seq_length):
        random_integer = random.randint(0,n_vocab-1)
        random_string += (int_to_char[random_integer])
        
    dataX+=data
    dataX+=random_string
    dataY+=[1]
    dataY+=[0]

dataX=np.reshape(np.array(dataX),(int(len(dataX)/seq_length),seq_length)) #reshape in [Anzahl, Länge einer Sequenz]
Y=np.array(dataY)

print("X:")
print(dataX)
print("")
print("Y:")
print(Y)

X:
[['M' 'ä' 'n' 'n' 'e']
 ['5' ' ' '/' 'a' '3']
 ['ä' 'n' 'n' 'e' 'r']
 ...
 [']' '-' 'd' 'l' "'"]
 ['i' 'c' 'h' 't' 'e']
 ['%' 'x' '1' 'D' '~']]

Y:
[1 0 1 ... 0 1 0]


5: Jeden Buchstaben in seine Id umwandeln

In [5]:

X = []
for i in range(len(dataX)):
  for o in range(len(dataX[i])):
    X+=[char_to_int[dataX[i][o]]]

X = np.reshape(X, (int(len(X)/seq_length), seq_length))
print(X)

[[ 46 102  77  77  68]
 [ 22   1  16  64  20]
 [102  77  77  68  81]
 ...
 [ 62  14  67  75   8]
 [ 72  66  71  83  68]
 [  6  87  18  37  93]]


6: One-hot-encoding durchführen.
Beim one-hot-encoding werden Werte aus vielen 0 und einer 1 dargestellt.<br>
Zb ["a","b","c"] wird zu [[0,0,1],[0,1,0],[1,0,0]]

In [6]:
X = to_categorical(X,num_classes=n_vocab)
# reshape in [Anzahl, Länge der Sequenz, Länge der one-hot-encode-vectoren]
X = np.reshape(X, (int(len(X)), seq_length, n_vocab))

7: Modell erstellen. Testweise kann als LSTM auch GRU verwendet werden

In [7]:
filepath="weights.hdf5"

model = Sequential()
model.add(LSTM(32, input_shape=(seq_length, n_vocab),return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
try:
  model.load_weights(filepath)
except:
    pass

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 32)             17920     
_________________________________________________________________
dropout (Dropout)            (None, 5, 32)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 42,882
Trainable params: 42,882
Non-trainable params: 0
_________________________________________________________________
None


8: Modell trainieren<br>Wenn du das Modell oben runtergeladen hast kannst du diesen Schritt überspringen

In [8]:
# checkpoint festlegen
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, Y, epochs=10, batch_size=128,verbose=1, callbacks=callbacks_list)

Epoch 1/10
6839/6839 [==============================] - 65s 5ms/step - loss: 0.0511

Epoch 00001: loss improved from inf to 0.02216, saving model to weights.hdf5
Epoch 2/10
6839/6839 [==============================] - 32s 5ms/step - loss: 0.0088

Epoch 00002: loss improved from 0.02216 to 0.00806, saving model to weights.hdf5
Epoch 3/10
6839/6839 [==============================] - 31s 5ms/step - loss: 0.0060

Epoch 00003: loss improved from 0.00806 to 0.00571, saving model to weights.hdf5
Epoch 4/10
6839/6839 [==============================] - 31s 5ms/step - loss: 0.0047

Epoch 00004: loss improved from 0.00571 to 0.00451, saving model to weights.hdf5
Epoch 5/10
6839/6839 [==============================] - 31s 5ms/step - loss: 0.0037

Epoch 00005: loss improved from 0.00451 to 0.00374, saving model to weights.hdf5
Epoch 6/10
6839/6839 [==============================] - 31s 5ms/step - loss: 0.0033

Epoch 00006: loss improved from 0.00374 to 0.00328, saving model to weights.hdf5
Epoch 7/

9: Auswertung

In [9]:
while True:
  eingabe = input("text: ")
  if(eingabe=="ende"):
    break
  n_chars = len(eingabe)
  print ("Buchatsben: ", n_chars)
  dataX = []
  for i in range(n_chars-seq_length):
    data=eingabe[i:i+seq_length];
    dataX+=data
  dataX=np.reshape(np.array(dataX),(int(len(dataX)/seq_length),seq_length))
  Xeingabe = []
  for i in range(len(dataX)):
    for o in range(len(dataX[i])):
      Xeingabe+=[char_to_int[dataX[i][o]]]
  Xeingabe = to_categorical(Xeingabe,num_classes=n_vocab)

  # reshape X to be [samples, time steps, features]
  Xeingabe = np.reshape(Xeingabe, (int(len(Xeingabe)/seq_length), seq_length, n_vocab))
  wahr = 0.0
  falsch = 0.0
  gesamt = 0
  for i in Xeingabe:
    gesamt+=1
    Xeingabe=np.reshape(np.array(i),(1,seq_length,n_vocab))
    res = model.predict(Xeingabe)
    
    wahr += res[0][1]
    falsch +=res[0][0]
  
  print("Bewertung Text: ",((wahr - falsch)/gesamt))
  print("")


text: ohibokjdbmfoybdpf
Buchatsben:  17
Bewertung Text:  -0.49505784984503237

text: Hallo, das ist ein Test
Buchatsben:  23
Bewertung Text:  0.9997712402930661

text: ende
